# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

Thu 2023-01-26 22:34:18.042117 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
Thu 2023-01-26 22:34:18.042552 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
Thu 2023-01-26 22:34:18.042597 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
Thu 2023-01-26 22:34:18.044518 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
Thu 2023-01-26 22:34:18.069155 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f0f328429b0>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
# ANSI escape codes for colors
CRESET    = '\33[0m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CVIOLET = '\33[35m'

def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    elif tag == "server":
        color = CVIOLET
    else:
        color = CGREEN
    print(f"{color}{message}{CRESET}")
        

class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
{"c": client_tiface, "s": server_tiface}

{'c': <__main__.LoggingTransportInterface at 0x7f0f326fe260>,
 's': <__main__.LoggingTransportInterface at 0x7f0f326bb8b0>}

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7f0f32759340>,
 's': <FilterInterface at 0x7f0f32759210>}

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

LTI(client)0: got transport notifications: []
Thu 2023-01-26 22:34:18.124330 [INFO ] - [F] About to send pings to servers: []
LTI(server)0: got transport notifications: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation for Pings

### As a client, add server as ping endpoint

In [10]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013)] }

In [11]:
await client_fiface.command_response(filter_cmd)

Thu 2023-01-26 22:34:18.253459 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013)] }


Accepted

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 12) to be received back from server before we know the latency.

In [12]:
time.sleep(3.0)
server_notifications = server_fiface.get_notifications()
server_notifications

Thu 2023-01-26 22:34:18.324318 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:18.523335 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:18.724093 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:18.924016 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:19.124299 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:19.323670 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:19.523887 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:19.724205 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:19.923626 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:20.123966 [INFO ] - [F] About to s

[]

In [13]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

Sleeping again....

In [14]:
time.sleep(7.0)
client_notifications = client_fiface.get_notifications()
client_notifications

Thu 2023-01-26 22:34:21.323874 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:21.523670 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:21.723493 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:21.923552 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:22.123751 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:22.323709 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:22.524167 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:22.724035 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:22.924003 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013)]
Thu 2023-01-26 22:34:23.124290 [INFO ] - [F] About to s

[]

In [15]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

LTI(client)104 skipped 103: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 10273940783305915484 } }] }
Thu 2023-01-26 22:34:28.315791 [TRACE] - [F<-A,C] New command: ClearPingEndpoints
Thu 2023-01-26 22:34:28.315816 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(127.0.0.1:62013)]
LTI(client)105: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI(client)106: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 0 } }
Thu 2023-01-26 22:34:28.315957 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 10273940783305915484 } }] }
Thu 2023-01-26 22:

Accepted

Thu 2023-01-26 22:34:28.323435 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:44678), took packet GetStatus { ping: PingPong { nonce: 10273940783305915484 } }
Thu 2023-01-26 22:34:28.323507 [INFO ] - [F] Sending Status packet PingPong { nonce: 10273940783305915484 } back to client Endpoint(127.0.0.1:44678)
Thu 2023-01-26 22:34:28.323690 [INFO ] - [F] About to send pings to servers: []
LTI(client)107: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI(server)328 skipped 327: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:44678), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 1 }, retry_interval: 0ns }], packets: [Status { pong: PingPong { nonce: 10273940783305915484 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }] }
Thu 2023-01-26 22:34:28.325732 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:44678), packet_infos

Thu 2023-01-26 22:34:40.724013 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:40.923597 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:41.123788 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:41.324040 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:41.523850 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:41.724019 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:41.924249 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:42.124295 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:42.323685 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:42.523554 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:42.724077 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:42.924241 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:34:43.123534 [INFO ] -

Thu 2023-01-26 22:35:01.323631 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:01.523733 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:01.723814 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:01.923424 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:02.123938 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:02.323942 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:02.523572 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:02.724139 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:02.923894 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:03.123405 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:03.323940 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:03.523271 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:35:03.724008 [INFO ] -

...

Thu 2023-01-26 22:36:03.323954 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:03.523827 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:03.723977 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:03.923404 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:04.124116 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:04.323407 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:04.524193 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:04.723460 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:04.924172 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:05.123640 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:05.323828 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:05.524064 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:05.724297 [INFO ] -

Thu 2023-01-26 22:36:24.123282 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:24.323632 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:24.523972 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:24.723598 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:24.923941 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:25.123541 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:25.324061 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:25.524029 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:25.723805 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:25.923457 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:26.123879 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:26.324091 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:26.524052 [INFO ] -

Thu 2023-01-26 22:36:44.724407 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:44.924209 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:45.123766 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:45.323685 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:45.523557 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:45.723798 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:45.923359 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:46.123958 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:46.323462 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:46.523569 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:46.724367 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:46.924134 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:36:47.123743 [INFO ] -

Thu 2023-01-26 22:37:05.524170 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:05.723533 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:05.923422 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:06.123926 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:06.323541 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:06.524405 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:06.724081 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:06.923657 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:07.123741 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:07.323612 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:07.523853 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:07.724044 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:07.923570 [INFO ] -

Thu 2023-01-26 22:37:26.324393 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:26.523966 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:26.724186 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:26.923735 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:27.124148 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:27.324291 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:27.523612 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:27.723695 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:27.923804 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:28.123314 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:28.323514 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:28.523922 [INFO ] - [F] About to send pings to servers: []
Thu 2023-01-26 22:37:28.723491 [INFO ] -